In [1]:
import LinearAlgebra
import Random
import BenchmarkTools
import StaticArrays
import NBInclude
import LinearAlgebra
import Distributions
import StatsBase
import Profile
import StatProfilerHTML

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1662


# sequence simulator

In [2]:
# this function takes the length as an argument to generate sequences
# note, this can also be used to generate a motif
function GenerateSequence(Length)
    return collect(Random.randstring("ACGT", Length))
end

GenerateSequence (generic function with 1 method)

In [3]:
# create a vector of vector{chars}, each with the same length
function GenerateSequences(NumberOfSequences, Length)
    return Sequences = map(i -> GenerateSequence(Length), 1:NumberOfSequences)
end

GenerateSequences (generic function with 1 method)

In [4]:
function Mutate!(Sequence, Distance)
    # return the sequence if the Distance is 0
    if Distance == 0
        return Sequence
    end
    # choose the sites to mutate
    posToMutate = StatsBase.sample(1:length(Sequence), Distance, replace = false)
    
    # pick the letters that can be used at each position
    basesAtPositions = [string(i) for i in Sequence[posToMutate]]

    lettersToFill = ["ACGT" for i in 1:length(Sequence)]
    
    # figure out what letters each positions can be changed to
    lettersToFill = map((x, y) -> replace(x, y => ""), lettersToFill, basesAtPositions)
    
    # select one character from each of these positions and place them where they should be
    # use only to cast the string to a character
    # https://stackoverflow.com/questions/59946081/julia-convert-string-to-char-or-convert-arraysubstringstring-1-to-char
    Sequence[posToMutate] = map(x -> only(Random.randstring(x, 1)), lettersToFill)
    
    return Sequence
end

Mutate! (generic function with 1 method)

In [5]:
# generate test data
# distance is the number of mutations in each implanted motif
function GenerateTestData_ld(NumberOfSequences, LengthMotif, LengthSequences, Distance)
    # do some error checking to make sure the values provided are valid
    # specifically, Disance <= LengthMotif
    if Distance > LengthMotif
        error("The Distance if larger than the LengthMotif.")
    end

    # LengthMotif <= LengthSequences
    if LengthMotif > LengthSequences
        error("The LengthMotif is longer than the LengthSequences")
    end
    
    # NumberOfSequences >= 2
    if NumberOfSequences <= 1
        error("The NumberOfSequences is 1, which is too small for motif detection")
    end

    # generate the input sequences
    sequences = GenerateSequences(NumberOfSequences, LengthSequences)
    
    # make the motif
    motif = GenerateSequence(LengthMotif)
    
    # generate a mutated motif for each sequence to implant
    motifs = [copy(motif) for i in 1:NumberOfSequences]
    
    # broadcast the mutated motif
    motifs .= Mutate!.(motifs, Distance)
    
    # now implant the mutated motifs and record where we implant it
    motifStarts = rand(1:(LengthSequences - LengthMotif + 1), NumberOfSequences)
    
    # now place the motifs into the sequences vector
    for i in 1:length(sequences)
        sequences[i][motifStarts[i]:(motifStarts[i] + LengthMotif - 1)] = motifs[i]
    end
    
    # now return all the things we may want later
    return (motif = motif, motifs_starts = motifStarts, motifs_implanted = motifs, sequences = sequences)
end

GenerateTestData_ld (generic function with 1 method)

# Score Function

In [6]:
# it is written in such a way that it is continuously defined
function Score(Starts_inp, Length=length(motif), Seqs=sequences; ln=false)
    maxScore = Length * length(Seqs)
    # MODIFY the inputs if they are out of bounds :-)
    ### TO DO: return a very large number of the starts are out of bounds
    if @inbounds (minimum(Starts_inp) < 1) | (maximum(Starts_inp) > (length(Seqs[1]) - Length + 1))
        return(maxScore)
    end
    # in the mean time, just fix out of bounds errors
    
    # generate the character matrix
    # then store the results inside of it
    #@inbounds @views seqsMatrix = mapreduce(permutedims, vcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp))
    @inbounds seqsMatrix = mapreduce(permutedims, vcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp))
    
    # allocate an array for the most common values in each column
    @inbounds mostCommon = Array{Char}(undef, size(seqsMatrix, 2))
    thisScore = 0
   
    # find the most common element in each column 
    for i in 1:size(seqsMatrix, 2)
        A = count(x -> x == 'A', @views @inbounds seqsMatrix[:,i])
        C = count(x -> x == 'C', @views @inbounds seqsMatrix[:,i])
        G = count(x -> x == 'G', @views @inbounds seqsMatrix[:,i])
        T = count(x -> x == 'T', @views @inbounds seqsMatrix[:,i])
        
        char_max = max(A, C, G, T)
        thisScore += char_max
        
        if char_max == A
            @inbounds mostCommon[i] = 'A'
        elseif char_max == C
            @inbounds mostCommon[i] = 'C'
        elseif char_max == G
            @inbounds mostCommon[i] = 'G'
        else
            @inbounds mostCommon[i] = 'T'
        end     
    end

    if ln
        return(-1 * log(maxScore - thisScore + 1))
    else
    # make the minimum (best) score 1 so we can compute the log
        return(maxScore - thisScore + 1)
    end
end

Score (generic function with 3 methods)

In [7]:
# it is written in such a way that it is continuously defined
function ConsensusMotif(Starts_inp, Length=length(motif), Seqs=sequences; ln=false)
    
    # generate the character matrix
    # then store the results inside of it
    #@inbounds @views seqsMatrix = mapreduce(permutedims, vcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp))
    @inbounds seqsMatrix = mapreduce(permutedims, vcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp))
    
    # allocate an array for the most common values in each column
    @inbounds mostCommon = Array{Char}(undef, size(seqsMatrix, 2))
    thisScore = 0
   
    # find the most common element in each column 
    for i in 1:size(seqsMatrix, 2)
        A = count(x -> x == 'A', @views @inbounds seqsMatrix[:,i])
        C = count(x -> x == 'C', @views @inbounds seqsMatrix[:,i])
        G = count(x -> x == 'G', @views @inbounds seqsMatrix[:,i])
        T = count(x -> x == 'T', @views @inbounds seqsMatrix[:,i])
        
        char_max = max(A, C, G, T)
        thisScore += char_max
        
        if char_max == A
            @inbounds mostCommon[i] = 'A'
        elseif char_max == C
            @inbounds mostCommon[i] = 'C'
        elseif char_max == G
            @inbounds mostCommon[i] = 'G'
        else
            @inbounds mostCommon[i] = 'T'
        end     
    end
    
    return(mostCommon)
end

ConsensusMotif (generic function with 3 methods)

In [8]:
# calculate a partial score in the plus and minus direction
# return positive if going in the positive direction increases the score
# return negative if going in the negative direction increases the score
# the value given is the sum score difference gained from going x-1 -> x -> x+1
#### TO DO: deal with edges of the search space
#### TO DO: speed up the update by only updating the score by a change in one dimension
# https://www2.atmos.umd.edu/~ekalnay/syllabi/AOSC614/NWP-CH03-2-2.pdf
function Grad(Starts_inp, seq, dt, k, Seqs)
    backward = copy(Starts_inp)
    forward = copy(Starts_inp)
    
    backward[seq] -= dt
    forward[seq] += dt
    
    #score1 = Score(Starts_inp) - Score(backward)
    #score2 = Score(forward) - Score(Starts_inp)
    return((Score(forward, k, Seqs; ln = true) - Score(backward, k, Seqs; ln = true))/(2*dt))
end

Grad (generic function with 1 method)

In [9]:
function CalcGrad(StartsInp, dt, k, Seqs)
    # make a vector copy we can modify
    backward = copy([i for i in StartsInp])
    forward = copy([i for i in StartsInp])
    grad = zeros(Float64, length(StartsInp))
    
    for seq in 1:length(StartsInp)
        backward[seq] -= dt
        forward[seq] += dt
        grad[seq] = (Score(forward, k, Seqs; ln = true) - Score(backward, k, Seqs; ln = true))/(2*dt)
    end
    
    return(grad)
end

CalcGrad (generic function with 1 method)

# HMC

In [28]:
function H(particle, velocity, k, Seqs)
    return(U(particle, k, Seqs) + 1/2 * velocity' * inv(LinearAlgebra.I) * velocity)
end

H (generic function with 1 method)

In [29]:
function U(particle, k, Seqs)
    return(-1 * Score(particle, k, Seqs; ln = true) )
end

U (generic function with 1 method)

# try to do HMC in place in memory

In [12]:
function leap!(particle, velocity, dt, iter, k, Seqs, i, parts, scores)
    #### To do: convert these vectors to staticarrays
    # initial particle position and velocity
    xnt = copy([i for i in particle])
    pnt = Float64.(copy(velocity))
    pn_t_dt = zeros(Float64, length(particle))
    pn_t_dt2 = zeros(Float64, length(particle))
    xn_t_dt = copy([i for i in particle])
    
    # iterate for the number of time steps desired
    # TODO: add stopping criterion
    for t in 1:iter
        # do the first half-step
        # update the particles momentum by looking at the previous moments
        # and adding half-ish the change in the potential energy
        pn_t_dt2 .= pnt .- (dt/2) .* CalcGrad(xnt, dt, k, Seqs)

        # update the particle position after t time
        # by using the new momentum, found in the previous half step
        
        # round this value to an integer, since our scorer only works
        # on discrete values
        ### TODO: add a bounds check here to prevent from going out of bounds
        # also, do something different besides rounding, like using the logic in add_vectors
        xn_t_dt .= Int.(round.(xnt .+ dt * inv(LinearAlgebra.I) * pn_t_dt2))
        
        # update the momentum again, this time by evaluating the gradient at the new point
        pn_t_dt .= pn_t_dt2 .- (dt/2) .* CalcGrad(xn_t_dt, dt, k, Seqs)
        
        # do a metropolis-hastings step
        
        # compute the ratio of hamiltonian new/old like on the wiki page      
        # if this value is off substantially, we will end up rejecting the sample
        α = min(1, (exp(-1 * H(xn_t_dt, pn_t_dt, k, Seqs)))/(exp(-1 * H(xnt, pnt, k, Seqs))))
        # quit if we're deviating substantially from the HMC path
        if α < 0.0001
            return 
        end
        if (minimum(xn_t_dt) < 1) | (maximum(xn_t_dt) > (length(Seqs[1]) - k + 1))
            break
        end
        
        # if our random number is less than alpha, accept the new proposal
        if rand(1)[1] <= α
            xnt .= xn_t_dt
            pnt .= pn_t_dt
            parts[i[1]] = StaticArrays.SVector{length(particle), Int64}(xnt)
            scores[i[1]] = Score(xnt, k, Seqs)
            i .+= 1
        end
        
        #### TODO:
        # check if we are at a boundary. if so, reverse the sign of the velocity for that dimension
    end
    #### TODO: also return the scores of each particle
    return
end

leap! (generic function with 1 method)

In [13]:
function make_sample!(particle, velocity, num, dt, iter, k, Seqs, parts, scores)
    # TODO: preallocate the vector for the particles
    # likely need these two vectors on from Algorithm, and just keep reusing the same vectors over and over
    # preallocate scores with a very large number probably, maybe the largest Int64?
    this_part = zeros(Int64, length(particle))
    this_velo = zeros(Int64, length(velocity))
    i = [1]
    # pre allocate the vector for the sample
    for n in 1:num
        this_part .= particle .+ velocity .+ StatsBase.sample(-5:5, length(particle)) ### TODO: initiate the starting position of the particle based on particle and velocity
        this_velo .= StatsBase.sample(-3:3, length(particle)) ### TODO: initiate the velocity
        # sample random starting points at particle + 
        leap!(this_part, this_velo, dt, iter, k, Seqs, i, parts, scores)
    end
    
    # handle the case where no sampling is done
    if length(scores) == 0
        return(parts, [], Inf)
    end
    minpos = argmin(scores)
    bestpart = parts[minpos]
    bestscore = scores[minpos]
    # return the sample, as well as the (particle, score) having the best score
    return(bestpart, bestscore)
end

make_sample! (generic function with 1 method)

In [14]:
parts = [StaticArrays.SVector{3, Int64}(zeros(3)) for i in 1:(30*30)]
scores = [typemax(Int64) for i in 1:(30*30)]

900-element Vector{Int64}:
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
                   ⋮
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807
 9223372036854775807

In [26]:
parts

900-element Vector{StaticArraysCore.SVector{3, Int64}}:
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 ⋮
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]
 [0, 0, 0]

In [15]:
@allocated parts = [StaticArrays.SVector{3, Int64}(zeros(3)) for i in 1:(30*30)]

3068708

In [16]:
@allocated begin
    for i in 1:length(parts)
        parts[i] = StaticArrays.SVector{3, Int64}(zeros(3))
    end
end

175504

In [ ]:
#Random.seed!(123)
#@allocated make_sample!(StaticArrays.SVector{3, Int64}(10, 10, 10), StaticArrays.SVector{3, Int64}(1, 1, 1), 30, 1, 30, 10, sequences, parts, scores)

# PSO

# try some different logic in add_vectors

In [17]:
function add_vectors(vec1, vec2, energy)
    # if vector to add is all zeros, just return the original vector
    if vec2 == zeros(length(vec1))
        return vec1
    end

    # find the norm of the first vector, which is the particle
    #added = (vec1 .+ vec2).*(LinearAlgebra.norm(vec1, 2)/(LinearAlgebra.norm(vec1, 2)+LinearAlgebra.norm(vec2, 2)))
    
    # current_sign = sign.(added)
    
    # get the remainder 
    # rounded_ver = abs.(added) .- floor.(abs.(added))
    
    # for the negative numbers, take 1 minus the value
    # rounded_ver[current_sign .== -1.0] .= 1 .- rounded_ver[current_sign .== -1.0]

    # get the remainder part
    # round_dir = ifelse.(rounded_ver .< rand(length(vec1)), RoundUp::RoundingMode, RoundDown::RoundingMode)
    # add the two vectors together
    # now round each value to get an integer array
    # get a random number for each position in the vector
    
    # round the vector
    # vec_temp = map((x, y) -> round(Int64, x, y), added, round_dir)
    vec_temp = vec1 .+ vec2
    norm_temp = LinearAlgebra.norm(vec_temp)
    
    # make the vector longer if the norm is smaller than the energy
    if norm_temp < energy
        while (LinearAlgebra.norm(vec_temp) < energy) & (LinearAlgebra.norm(vec_temp) > 0)
            # pick a random position in the vector
            rand_pos = rand(1:length(vec_temp))
            # now subtract its sign from that position
            vec_temp[rand_pos] += sign(vec_temp[rand_pos])
        end
    # make the vector shorter if the norm is larger than the energy
    elseif norm_temp > energy
        while (LinearAlgebra.norm(vec_temp) > energy) & (LinearAlgebra.norm(vec_temp) > 0)
            # pick a random position in the vector
            rand_pos = rand(1:length(vec_temp))
            # now add its sign from that position
            vec_temp[rand_pos] -= sign(vec_temp[rand_pos])
        end
    end
    
    # if any of the dimensions are zeros - try adding 1
    
    vec_out = StaticArrays.SVector{length(vec1), Int64}(vec_temp)
    
    # if all equal to zero, just use the sign of vec2
    if vec_out == zeros(length(vec1))
        return sign.(vec2) .* StatsBase.sample(1:3, length(vec1))
    else
        return vec_out
    end
    ## todo: deal with situations where the norm of vec1 is very close tozero
end

add_vectors (generic function with 1 method)

In [18]:
add_vectors([4, 4, 4], [-2, 1, 2], 7)

3-element StaticArraysCore.SVector{3, Int64} with indices SOneTo(3):
 0
 4
 5

In [27]:
GetFinalParticle(sequences, 10, ['G', 'A', 'T', 'C', 'C', 'T', 'G', 'G', 'T', 'A'])

10-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [22]:
sequences = GenerateSequences(10,10)

10-element Vector{Vector{Char}}:
 ['C', 'C', 'T', 'T', 'C', 'A', 'G', 'G', 'C', 'T']
 ['A', 'G', 'C', 'C', 'G', 'T', 'G', 'G', 'G', 'A']
 ['C', 'C', 'T', 'A', 'C', 'T', 'T', 'A', 'T', 'G']
 ['A', 'C', 'A', 'G', 'C', 'C', 'A', 'A', 'T', 'G']
 ['C', 'T', 'A', 'A', 'A', 'C', 'G', 'A', 'G', 'C']
 ['T', 'T', 'T', 'T', 'A', 'A', 'C', 'T', 'A', 'T']
 ['C', 'G', 'G', 'T', 'C', 'C', 'G', 'G', 'G', 'A']
 ['T', 'G', 'C', 'G', 'G', 'G', 'C', 'C', 'A', 'T']
 ['C', 'T', 'G', 'G', 'G', 'C', 'A', 'C', 'G', 'C']
 ['G', 'C', 'A', 'G', 'T', 'T', 'T', 'A', 'A', 'T']

In [20]:
function GetFinalParticle(seqs, k, consensus)
    # go over all the sequences
    out_vec = zeros(Int64, length(seqs))
    
    for i in 1:length(sequences)
        # score the starting positions in each sequence
        dist = map(start -> sum(consensus .== seqs[i][start:(start + k - 1)]), 1:(length(seqs[i]) - k + 1))
        out_vec[i] = argmax(dist)
    end
    return(out_vec)
end

GetFinalParticle (generic function with 1 method)

In [24]:
function Algorithm(seqs, k, dims = 3, nparticles = 10, maxiter = 500)
    debug = false
    # generate the starting points for the algorithm
    # store these in a matrix, where each column is a particle, and each row is a dimension in the search space
    particles = [StaticArrays.SVector{dims, Int64}(rand(1:(length(seqs[1]) - k + 1), dims)) for i in 1:nparticles]
    velocities = [StaticArrays.SVector{dims, Int64}(rand(-5:5, dims)) for i in 1:nparticles]
    energies = Int.(round.([LinearAlgebra.norm(i) for i in velocities]))

    scores = zeros(Int64, nparticles)
    
    # create a set to store the points that we have visited, to avoid looking at the same points repeatedly
    visited = Set{StaticArrays.SVector{dims, Int64}}()
    
    # give a size hint for the set of visited points, hopefully preventing it from growing repeatedly
    sizehint!(visited, nparticles * maxiter)
    
    iter = 1
    
    best_particle = zeros(Int64, dims)
    global_optimum = Inf

    terminated = 0
    
    # reuse the same value for sign over and over
    sign_vec = zeros(Int64, dims)
    
    # create the vectors for storing the samples
    parts = [StaticArrays.SVector{dims, Int64}(zeros(dims)) for i in 1:(30*30)]
    scores_samp = [typemax(Int64) for i in 1:(30*30)]
    idx = 1
    # loop until convergence
    
    if debug
        for p in 1:nparticles
            println(particles[p], velocities[p])
        end

        println()
    end
    
    while iter <= maxiter
        # wrap particles around that have gone out of bounds
        counter = 0
        # "wiggle" each particle position if it's already been visited, or if its out of bounds
        # for each of these, try wiggling
        for part in findall(x -> (x in visited) | (minimum(x) < 1) | (maximum(x) > (length(seqs[1]) - k + 1)), particles)
            counter += 1
            newpart = copy(particles[part])
            tries = 1
            while (tries < 40) & (newpart in visited)
                dist = Int(floor(tries/4 + 1))
                newpart = particles[part] .+ (rand(1:dist, dims) .* rand([-1, 0, 1], dims))
                tries += 1
            end
            particles[part] = newpart
        end
        

        # compute the current scores of each particle
        #scores .= map(x -> Score(x, k, seqs), particles)
        scores .= Score.(particles, (k,), (seqs,))
        
        # find the current best particle
        best_score = findmin(scores)
 
        # if this score is better than the current best score, update
        if best_score[1] < global_optimum
            global_optimum = copy(best_score[1])
            best_particle .= copy(particles[best_score[2]])
        end
    
        # for each of the current vectors
            # sample nearby positions and run HMC for 30 steps for each of 30 sampled particles
            # sample such that particles are more likely to start in the direction the particle is heading and in the direction of the best particle

        for i in 1:length(particles)
            # make sure the samples are set to be empty
            for i in 1:idx
                parts[i] = StaticArrays.SVector{dims, Int64}(zeros(dims))
                scores_samp[i] = typemax(Int64)
            end
            
            # try adding the direction rather than the sign
            sign_vec .= sign.(best_particle .- particles[i])
            # sign_vec .= best_particle .- velocities[i]
            # update the velocity first to be more in the direction of the best particle
            # keep constant energy/total velocity norm for each particle
            velocities[i] = add_vectors(velocities[i], sign_vec, energies[i])

            # draw the sample
            # make_sample!(particle, velocity, num, dt, iter, k, Seqs, parts, scores)
            bestpart_sample, bestscore_sample = make_sample!(particles[i], velocities[i], 30, 1, 30, k, seqs, parts, scores_samp)
            idx = findfirst(isequal(zeros(dims)), parts)
            
            # check the sample to see if we found a really good position to aim towards
            if bestscore_sample < global_optimum
                global_optimum = copy(bestscore_sample)
                best_particle .= copy(bestpart_sample)
            end

            # handle the case where sampling fails since we're on the edge of the search space
            if idx > 1
                # pick a random term from the sample and update the particle position
                @inbounds particles[i] = StatsBase.sample(@views parts[1:(idx - 1)])
                # now update to be in the direction of the best particle
                
            end
        end        
        
        # save the new locations we've visited
        map(x -> push!(visited, x), particles)

        # update the particle positions based on the velocities
        
        # convergence criteria: quit when idx is full for ten iterations in a row
        if counter == nparticles
            terminated += 1
        end
        
        if terminated == 10
            iter += maxiter
        end

        iter += 1
        
        if debug
            for p in 1:nparticles
                println(particles[p], velocities[p], Score(particles[p], k, seqs))
            end
            println(global_optimum, best_particle)
            println()
        end
    end 
    
    # may want to repeat this step multiple times to refine motif
    consensus_motif = ConsensusMotif(best_particle, k, seqs)
    final_particle = GetFinalParticle(seqs, k, consensus_motif)
    final_score = Score(final_particle, k, seqs)
    
    consensus_motif = ConsensusMotif(final_particle, k, seqs)
    
    # add a shift step to see if moving up or down helps the score
    ## finishing step
    ## calculate the position weight matrix, and find the highest scoring motif in each sequence.
    
    
    # also return the consensus motif
    return(final_score, final_particle, consensus_motif)
end

Algorithm (generic function with 4 methods)

# Helpful Output formatting

In [25]:
function print_motifs(Starts_Inp, Seqs, k)
    for i in 1:length(Starts_Inp)
        println(join(Seqs[i][(Starts_Inp[i]):(Starts_Inp[i] + k - 1)]))
    end
    return
end

print_motifs (generic function with 1 method)

# Unit Testing

In [ ]:
# test add_vectors!

In [ ]:
# check if out of bounds is working in Score
# motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 14, 30, 0)
# Score([1, 1, 0], sequences, 14)
# Score([1, 1, 18], sequences, 14)

# Hyperparameter Tuning

# Benchmarking

In [ ]:
# set the seed
Random.seed!(101)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 10, 30, 0)

In [ ]:
test_vec = Algorithm(sequences, 10, size(sequences, 1), 3, 10)

In [ ]:
Random.seed!(123)
@allocated Algorithm(sequences, 10, size(sequences, 1), 10, 10)

In [ ]:
BenchmarkTools.@btime Algorithm(sequences, 10, size(sequences, 1), 30, 1000)

In [ ]:
StatProfilerHTML.@profilehtml Algorithm(sequences, 10, size(sequences, 1), 30, 500)

In [ ]:
# set the seed
Random.seed!(100)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(50, 10, 250, 2)

In [ ]:
StatProfilerHTML.@profilehtml Algorithm(sequences, 10, size(sequences, 1), 30, 100)

# Visualizing HMC sampling in 2D